In [1]:
import warnings
import pandas as pd

from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import impute

In [2]:
from config import input_file, features_file

if __name__ == '__main__':

	d = pd.read_csv( input_file, header = None )

	columns = list( d.columns )
	columns.pop()
	columns.append( 'target' )
	d.columns = columns

	y = d.target
	d.drop( 'target', axis = 1, inplace = True )

	d = d.stack()
	d.index.rename([ 'id', 'time' ], inplace = True )
	d = d.reset_index()
	
	print (len (d) )
	d.head()
	
	# doesn't work too well
	with warnings.catch_warnings():
		warnings.simplefilter( "ignore" )
		f = extract_features( d, column_id = "id", column_sort = "time" )

	#c:\usr\anaconda\lib\site-packages\scipy\signal\spectral.py:1633: 
	# UserWarning: nperseg = 256 is greater than input length  = 152, using nperseg = 152

	# Feature Extraction: 20it [22:33, 67.67s/it]
	
	impute( f )
	assert f.isnull().sum().sum() == 0
	
	f['y'] = y
	f.to_csv( features_file, index = None )

462


Feature Extraction: 100%|██████████| 26/26 [00:00<00:00, 27.88it/s]
/home/nitin/anaconda3/lib/python3.7/site-packages/tsfresh/utilities/dataframe_functions.py:173: RuntimeWarning: The columns ['0__agg_linear_trend__attr_"intercept"__chunk_len_10__f_agg_"max"'
 '0__agg_linear_trend__attr_"intercept"__chunk_len_10__f_agg_"mean"'
 '0__agg_linear_trend__attr_"intercept"__chunk_len_10__f_agg_"min"'
 '0__agg_linear_trend__attr_"intercept"__chunk_len_10__f_agg_"var"'
 '0__agg_linear_trend__attr_"intercept"__chunk_len_50__f_agg_"max"'
 '0__agg_linear_trend__attr_"intercept"__chunk_len_50__f_agg_"mean"'
 '0__agg_linear_trend__attr_"intercept"__chunk_len_50__f_agg_"min"'
 '0__agg_linear_trend__attr_"intercept"__chunk_len_50__f_agg_"var"'
 '0__agg_linear_trend__attr_"rvalue"__chunk_len_10__f_agg_"max"'
 '0__agg_linear_trend__attr_"rvalue"__chunk_len_10__f_agg_"mean"'
 '0__agg_linear_trend__attr_"rvalue"__chunk_len_10__f_agg_"min"'
 '0__agg_linear_trend__attr_"rvalue"__chunk_len_10__f_agg_"var"'
 

In [3]:

import pandas as pd
from tsfresh import extract_features, select_features

from config import *


In [4]:
if __name__ == '__main__':

	print ("loading {}".format( features_file ))
	features = pd.read_csv( features_file )

	train_x = features.iloc[:validation_split_i].drop( 'y', axis = 1 )
	test_x = features.iloc[validation_split_i:].drop( 'y', axis = 1 )

	train_y = features.iloc[:validation_split_i].y
	test_y = features.iloc[validation_split_i:].y

	print ("selecting features...")
	train_features_selected = select_features( train_x, train_y, fdr_level = fdr_level )

	print ("selected {} features.".format( len( train_features_selected.columns )))

	train = train_features_selected.copy()
	train['y'] = train_y

	test = test_x[ train_features_selected.columns ].copy()
	test['y'] = test_y
	
	#

	print ("saving {}".format( train_file ))
	train.to_csv( train_file, index = None )

	print ("saving {}".format( test_file ))
	test.to_csv( test_file, index = None )
	

loading data/features.csv
selecting features...
selected 0 features.
saving data/train.csv
saving data/test.csv
